<a href="https://colab.research.google.com/github/matshch/iu5_bi_course/blob/master/lab2/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>